## **Project 2: Hyperparameter Optimization**

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten, Conv2D, Dropout, BatchNormalization
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing import image

In [2]:
trainDIR = '/workspace/seg_train/seg_train/'
testDIR = '/workspace/seg_test/seg_test/'
predDIR = '/workspace/seg_pred/seg_pred'

In [3]:
def get_images(directory):
    Images=[]
    Labels=[]
    
    for labels in os.listdir(directory):
        if labels == 'glacier':
            label = 0
        elif labels == 'sea':
            label = 1
        elif labels == 'buildings':
            label = 2
        elif labels == 'forest':
            label = 3
        elif labels == 'mountain':
            label = 4
        elif labels == 'street':
            label = 5
        
        for file in os.listdir(directory+labels):
            im =  image.load_img(directory+labels+r'/'+file, target_size=(150,150))
            img = image.img_to_array(im)
            Images.append(img)
            Labels.append(label)
    
    Images, Labels = sklearn.utils.shuffle(Images, Labels, random_state=4)
    return Images, Labels

In [4]:
Images, Labels = get_images(trainDIR)
Images = np.array(Images)
Labels = np.array(Labels)

In [5]:
train = Images[:3000]
train_labels=Labels[:3000]

In [6]:
test, test_labels = get_images(testDIR)
test = np.array(test)
test_labels = np.array(test_labels)

In [7]:
test.shape

(3000, 150, 150, 3)

In [8]:
def create_model(lr=0.01, nl1 = 1, nl2 = 1, dropout = 0.5,
                 nn1 = 32, nn2 = 500, batch_size=32):
    
    opt = keras.optimizers.Adam(lr=lr)
                                                     
    model = Sequential()
    
    for i in range(nl1):
        if i == 0:
            model.add(Convolution2D(nn1,(3,3), input_shape=(150,150,3), activation='relu'))
            model.add(MaxPooling2D((2,2)))
            model.add(BatchNormalization())
        else:
            model.add(Convolution2D(nn1,(3,3), activation='relu'))
            model.add(MaxPooling2D((2,2)))
            model.add(BatchNormalization())
        nn1/=2
    model.add(Flatten())
        
    for i in range(nl2):
        model.add(Dense(nn2, activation='relu'))
        model.add(BatchNormalization())
        nn2/=2
    
    model.add(Dropout(dropout))   
    model.add(Dense(6, activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model


In [9]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# model class to use in the scikit random search CV 
model = KerasClassifier(build_fn=create_model, epochs=5, validation_split=0.25)

In [10]:
# learning rate parameters
lr=[1e-2, 1e-3]

# number of neurons in each layer
nn1 = [64,32]
nn2 = [1000,500]

# number of convolutional and maxpooling layers
nl1 = [1,2]

# number of dense layers (exlcusing output layer)
nl2 = [1,2]

# batch_sizes
batch_size=[64,32]

# dropouts

dropout = [0,0.5]

# dictionary summary
param_grid = dict(nn1= nn1, nn2 = nn2, lr=lr, nl1=nl1, nl2=nl2, batch_size=batch_size, dropout = dropout)

In [11]:
from sklearn.model_selection import RandomizedSearchCV, KFold

grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid, n_iter=3, n_jobs=1)

In [12]:
grid_result = grid.fit(train, train_labels)

Epoch 1/5
24/24 [==============================] - 1s 56ms/step - loss: 1.9144 - accuracy: 0.4720 - val_loss: 95.2595 - val_accuracy: 0.2260
Epoch 2/5
24/24 [==============================] - 1s 23ms/step - loss: 1.1177 - accuracy: 0.5887 - val_loss: 27.2919 - val_accuracy: 0.3140
Epoch 3/5
24/24 [==============================] - 0s 21ms/step - loss: 0.8426 - accuracy: 0.6747 - val_loss: 7.9727 - val_accuracy: 0.2960
Epoch 4/5
24/24 [==============================] - 0s 20ms/step - loss: 0.7366 - accuracy: 0.7087 - val_loss: 7.3003 - val_accuracy: 0.2100
Epoch 5/5
16/16 [==============================] - 0s 9ms/step - loss: 9.4917 - accuracy: 0.2580
Epoch 1/5
24/24 [==============================] - 1s 29ms/step - loss: 1.9359 - accuracy: 0.4627 - val_loss: 130.2158 - val_accuracy: 0.3320
Epoch 2/5
24/24 [==============================] - 0s 20ms/step - loss: 1.1416 - accuracy: 0.5873 - val_loss: 127.6157 - val_accuracy: 0.1560
Epoch 3/5
24/24 [==============================] - 0s 20m

In [13]:
print(grid_result.best_params_)

{'nn2': 1000, 'nn1': 64, 'nl2': 2, 'nl1': 2, 'lr': 0.001, 'dropout': 0, 'batch_size': 32}


In [14]:
best_model = grid_result.best_estimator_.model

In [15]:
best_model.evaluate(test,test_labels)

94/94 [==============================] - 1s 7ms/step - loss: 1.2645 - accuracy: 0.6343


[1.2645362615585327, 0.6343333125114441]